In [1]:
from keras_tuner import HyperModel
import keras_tuner as kt

import numpy as np
import pandas as pd
#!pip install keras_tuner
#import keras_tuner

import keras
from keras import models, layers, metrics, Sequential
from keras.layers import Dense, BatchNormalization, Dropout
import tensorflow as tf
from keras.regularizers import l1,l2,l1_l2
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Input, Dense, Multiply, Concatenate
from tensorflow.keras.models import Model

import tensorflow as tf
import random

import keras_tuner
from keras_tuner import HyperModel
import keras_tuner as kt

from sklearn.preprocessing import MinMaxScaler

2023-09-27 17:32:55.187097: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [185]:
# Import Dataset
firm_df = pd.read_pickle('firm_df49.pkl')
firm_df.index = pd.to_datetime(firm_df.index).to_period('M')

# macro-economic variables
# There is one features which has 37 missing values but before year 1980
macro_df = pd.read_pickle('macro_df.pkl')
macro_df.index = pd.to_datetime(macro_df.index, format='%m/%d/%Y')
macro_df.rename(columns={"ep": "ep_macro"}, inplace=True)
macro_index = macro_df.index[1:]
macro_new = macro_df.iloc[:-1, :]
macro_new.index = macro_index
macro_new.index = macro_new.index.to_period('M')

# Scale macro-features
from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler(feature_range=(-0.5, 0.5))
#macro_scaled = scaler.fit_transform(macro_new)
#macro_scaled_df = pd.DataFrame(macro_scaled, columns=macro_new.columns)
#macro_scaled_df.index = macro_new.index

# merged data 1
merged_df = pd.merge(firm_df, macro_new, left_on='jdate', right_on='sasdate', how='inner')
merged_df.index = firm_df.index

In [7]:
char_core = ['acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv',
       'chmom','chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa',
       'ill','indmom','invest','lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr',
       'orgcap','pchgm_pchsale','pchsale_pchinvt','pchsale_pchrect','pchsale_pchxsga',
       'retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp','std_dolvol','std_turn','turn']

info_list = ['fyear','year','jyear','permno','ticker','comnam','exchcd','exchname','siccd',
       'indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]

macro_col = ['RPI', 'W875RX1', 'DPCERA3M086SBEA', 'CMRMTSPLx', 'RETAILx','INDPRO', 'IPFPNSS', 'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD',
       'IPBUSEQ', 'IPMAT', 'IPDMAT', 'IPNMAT', 'IPMANSICS', 'IPB51222S','IPFUELS', 'CUMFNS', 'HWI', 'HWIURATIO', 'CLF16OV', 'CE16OV',
       'UNRATE', 'UEMPMEAN', 'UEMPLT5', 'UEMP5TO14', 'UEMP15OV','UEMP15T26', 'UEMP27OV', 'CLAIMSx', 'PAYEMS', 'USGOOD',
       'CES1021000001', 'USCONS', 'MANEMP', 'DMANEMP', 'NDMANEMP','SRVPRD', 'USTPU', 'USWTRADE', 'USTRADE', 'USFIRE', 'USGOVT',
       'CES0600000007', 'AWOTMAN', 'AWHMAN', 'HOUST', 'HOUSTNE','HOUSTMW', 'HOUSTS', 'HOUSTW', 'PERMIT', 'PERMITNE', 'PERMITMW',
       'PERMITS', 'PERMITW', 'AMDMNOx', 'ANDENOx', 'AMDMUOx', 'BUSINVx','ISRATIOx', 'M1SL', 'M2SL', 'M2REAL', 'BOGMBASE', 'TOTRESNS',
       'NONBORRES', 'BUSLOANS', 'REALLN', 'NONREVSL', 'CONSPI', 'S&P 500','S&P: indust', 'S&P div yield', 'S&P PE ratio', 'FEDFUNDS',
       'CP3Mx', 'TB3MS', 'TB6MS', 'GS1', 'GS5', 'GS10', 'AAA', 'BAA','COMPAPFFx', 'TB3SMFFM', 'TB6SMFFM', 'T1YFFM', 'T5YFFM', 'T10YFFM',
       'AAAFFM', 'BAAFFM', 'TWEXAFEGSMTHx', 'EXSZUSx', 'EXJPUSx','EXUSUKx', 'EXCAUSx', 'WPSFD49207', 'WPSFD49502', 'WPSID61',
       'WPSID62', 'OILPRICEx', 'PPICMM', 'CPIAUCSL', 'CPIAPPSL','CPITRNSL', 'CPIMEDSL', 'CUSR0000SAC', 'CUSR0000SAD',
       'CUSR0000SAS', 'CPIULFSL', 'CUSR0000SA0L2', 'CUSR0000SA0L5','PCEPI', 'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA',
       'CES0600000008', 'CES2000000008', 'CES3000000008', 'UMCSENTx','DTCOLNVHFNM', 'DTCTHFNM', 'INVEST', 'VIXCLSx', 'dp', 'ep_macro', 'b/m',
       'ntis', 'tbl', 'tms', 'dfy', 'svar']

macro_core =  ['dp', 'ep_macro', 'b/m','ntis', 'tbl', 'tms', 'dfy', 'svar']

def datasplit(df, dataset):
    df_train = df[df.index < pd.Period((str(1995)+"-1"),freq='M')]
    df_val = df[(df.index >= pd.Period((str(1995)+"-1"),freq='M')) & (df.index < pd.Period((str(2000)+"-1"),freq='M'))]
    df_test = df[df.index >= pd.Period((str(2000)+"-1"),freq='M')]
    if dataset == "firm":
        X_train = df_train[char_core]
        X_val = df_val[char_core]
        X_test = df_test[char_core]
    elif dataset == 'macro':
        X_train = df_train[macro_col]
        X_val = df_val[macro_col]
        X_test = df_test[macro_col]
    elif dataset == "firm_macro":
        X_train = df_train[char_core+macro_col]
        X_val = df_val[char_core+macro_col]
        X_test = df_test[char_core+macro_col]
    elif dataset == "firm_macro8":
        X_train = df_train[char_core+macro_core]
        X_val = df_val[char_core+macro_core]
        X_test = df_test[char_core+macro_core] 
    y_train = df_train['predicted_return']
    y_val = df_val['predicted_return']
    y_test = df_test['predicted_return']
    return (X_train, y_train, X_val, y_val, X_test, y_test, )

In [10]:
# HyperParameter Tuning
class MyHyperModel(HyperModel):
    def __init__(self, input_shape, l1_reg, l2_reg, layer, dropout, lr, batch_size, random_seed=None,):
        self.input_value = input_shape
        self.l1_reg = l1_reg
        self.l2_reg = l2_reg
        self.layer = layer
        self.dropout = dropout
        self.lr = lr
        self.batch_size = batch_size
        self.acti = 'relu'
        self.epoch = 200
        self.patience = 3

    if random_seed is not None:
        random.seed(random_seed)
        np.random.seed(random_seed)
        tf.random.set_seed(random_seed)
    
    def my_metric_fn(self, y_true, y_pred):
        num = tf.reduce_mean(tf.square(y_true - y_pred))
        den = tf.reduce_mean(tf.square(y_true - tf.zeros_like(y_true)))
    return 1 - num / den

    def call_existing_code(self, l1_reg, l2_reg, dropout, L, lr):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.input_value, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg, l2=l2_reg), activation=self.acti))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
        for i in range(L - 1):
            model.add(Dense(int(32/2**i), kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg, l2=l2_reg), activation=self.acti))
            model.add(Dropout(dropout))
            model.add(BatchNormalization())
        model.add(Dense(1))
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[self.my_metric_fn])
        return model
    
    def build(self, hp, *args, **kwargs):
        l1_reg = hp.Choice("l1_ratio", self.l1_reg)
        l2_reg = hp.Choice("l2_ratio", self.l2_reg)
        layer = hp.Choice("layer", self.layer)
        dropout = hp.Choice("dropout", self.dropout)
        lr = hp.Choice("learning_rate", self.lr)
        model = self.call_existing_code(
            l1_reg=l1_reg, l2_reg=l2_reg, L=layer, dropout=dropout, lr=lr)
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", self.batch_size),
            epochs=self.epoch,
            **kwargs,)


In [8]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(firm_df, dataset = "firm")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 49) (88347,) (31851, 49) (31851,) (90007, 49) (90007,)


In [11]:
random_seed = 1120
tuner = kt.RandomSearch(
    MyHyperModel(input_shape=X_train.shape[1],
                 l1_reg = [0.,],
                 l2_reg = [0.1,],
                 layer = [4],
                 dropout = [0.,],
                 lr = [0.0001],
                 batch_size = [128,],
                 random_seed=random_seed,
                 ),
    objective="val_loss",

    max_trials=100,
    directory="my_dir",
    overwrite=True,
    project_name="tune_hypermodel",
)
tuner.search(X_train, y_train, validation_data=(X_val, y_val),
       callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])

# Print best parameters
best_hps = tuner.get_best_hyperparameters(5)
hps1_dic = best_hps[0].values
print(hps1_dic)

Trial 1 Complete [00h 01m 20s]
val_loss: 0.014552339911460876

Best val_loss So Far: 0.014552339911460876
Total elapsed time: 00h 01m 20s
INFO:tensorflow:Oracle triggered exit
{'l1_ratio': 0.0, 'l2_ratio': 0.1, 'layer': 4, 'dropout': 0.0, 'learning_rate': 0.0001, 'batch_size': 128}


In [12]:
# 10 Trials for Prediction
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
best_hps = tuner.get_best_hyperparameters(5)
print(best_hps[0].values)
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = tuner.get_best_models(3)[0]
    best_model.fit(x=x_all, y=y_all, epochs=20,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(best_model.evaluate(X_test,y_test))
    y_hat = best_model.predict(X_test).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

{'l1_ratio': 0.0, 'l2_ratio': 0.1, 'layer': 4, 'dropout': 0.0, 'learning_rate': 0.0001, 'batch_size': 128}
Epoch 1/20
3757/3757 [==============================] - 13s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0013
Epoch 2/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0013
Epoch 3/20
3757/3757 [==============================] - 12s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0012
Epoch 4/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0014
Epoch 5/20
3757/3757 [==============================] - 12s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0014
Epoch 6/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0016
Epoch 7/20
2813/2813 [==============================] - 6s 2ms/step - loss: 0.0157 - my_metric_fn: -0.0063
[0.01568944752216339, -0.0063271671533584595]
2813/2813 [==============================] - 5s 2ms/step

Epoch 1/20
3757/3757 [==================

2813/2813 [==============================] - 5s 2ms/step

Epoch 1/20
3757/3757 [==============================] - 13s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0012
Epoch 2/20
3757/3757 [==============================] - 13s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0015
Epoch 3/20
3757/3757 [==============================] - 17s 4ms/step - loss: 0.0128 - my_metric_fn: 0.0013
Epoch 4/20
3757/3757 [==============================] - 18s 5ms/step - loss: 0.0128 - my_metric_fn: 0.0013
Epoch 5/20
2813/2813 [==============================] - 7s 2ms/step - loss: 0.0156 - my_metric_fn: 0.0069
[0.01563907228410244, 0.006861308589577675]
2813/2813 [==============================] - 6s 2ms/step

0.006681554282631508


In [13]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(merged_df, dataset = "firm_macro8")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 57) (88347,) (31851, 57) (31851,) (90007, 57) (90007,)


In [14]:
random_seed = 1120
tuner = kt.RandomSearch(
    MyHyperModel(input_shape=X_train.shape[1],
                 l1_reg = [0.,],
                 l2_reg = [0.1,],
                 layer = [4],
                 dropout = [0.,],
                 lr = [0.0001],
                 batch_size = [128,],
                 random_seed=random_seed,
                 ),
    objective="val_loss",

    max_trials=100,
    directory="my_dir",
    overwrite=True,
    project_name="tune_hypermodel",
)
tuner.search(X_train, y_train, validation_data=(X_val, y_val),
       callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])

# Print best parameters
best_hps = tuner.get_best_hyperparameters(5)
hps1_dic = best_hps[0].values
print(hps1_dic)

Trial 1 Complete [00h 01m 13s]
val_loss: 0.014552349224686623

Best val_loss So Far: 0.014552349224686623
Total elapsed time: 00h 01m 13s
INFO:tensorflow:Oracle triggered exit
{'l1_ratio': 0.0, 'l2_ratio': 0.1, 'layer': 4, 'dropout': 0.0, 'learning_rate': 0.0001, 'batch_size': 128}


In [15]:
# 10 Trials for Prediction
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
best_hps = tuner.get_best_hyperparameters(5)
print(best_hps[0].values)
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = tuner.get_best_models(3)[0]
    best_model.fit(x=x_all, y=y_all, epochs=20,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(best_model.evaluate(X_test,y_test))
    y_hat = best_model.predict(X_test).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

{'l1_ratio': 0.0, 'l2_ratio': 0.1, 'layer': 4, 'dropout': 0.0, 'learning_rate': 0.0001, 'batch_size': 128}
Epoch 1/20
3757/3757 [==============================] - 12s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0013
Epoch 2/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0013
Epoch 3/20
3757/3757 [==============================] - 10s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0012
Epoch 4/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0014
Epoch 5/20
3757/3757 [==============================] - 15s 4ms/step - loss: 0.0128 - my_metric_fn: 0.0014
Epoch 6/20
3757/3757 [==============================] - 13s 4ms/step - loss: 0.0128 - my_metric_fn: 0.0016
Epoch 7/20
2813/2813 [==============================] - 5s 2ms/step - loss: 0.0157 - my_metric_fn: 0.0025
[0.015724996104836464, 0.0024774575140327215]
2813/2813 [==============================] - 5s 2ms/step

Epoch 1/20
3757/3757 [===================

2813/2813 [==============================] - 5s 2ms/step

Epoch 1/20
3757/3757 [==============================] - 16s 4ms/step - loss: 0.0128 - my_metric_fn: 0.0012
Epoch 2/20
3757/3757 [==============================] - 13s 4ms/step - loss: 0.0128 - my_metric_fn: 0.0015
Epoch 3/20
3757/3757 [==============================] - 12s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0013
Epoch 4/20
3757/3757 [==============================] - 13s 4ms/step - loss: 0.0128 - my_metric_fn: 0.0013
Epoch 5/20
2813/2813 [==============================] - 6s 2ms/step - loss: 0.0156 - my_metric_fn: 0.0069
[0.01563958078622818, 0.006901421118527651]
2813/2813 [==============================] - 5s 2ms/step

0.0057806805528451966


In [186]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(merged_df, dataset = "firm_macro")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 183) (88347,) (31851, 183) (31851,) (90007, 183) (90007,)


In [188]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true - tf.zeros_like(y_true)))
    return 1 - num / den

def call_existing_code(input_value):
    L = 4
    l1_reg= 0.
    l2_reg= 0.1
    dropout =0.0
    lr =0.0001
    acti='relu'
    model = Sequential()
    model.add(Dense(units=64, input_dim=input_value, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg, l2=l2_reg), activation=acti))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    for i in range(L - 1):
        model.add(Dense(int(32/2**i), kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg, l2=l2_reg), activation=acti))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
    model.add(Dense(1))
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[my_metric_fn])
    return model

model = call_existing_code(input_value=X_train.shape[1])

In [189]:
random_seed = 1120
tuner = kt.RandomSearch(
    MyHyperModel(input_shape=X_train.shape[1],
                 l1_reg = [0.,],
                 l2_reg = [0.1,],
                 layer = [4],
                 dropout = [0.,],
                 lr = [0.0001],
                 batch_size = [128,],
                 random_seed=random_seed,
                 ),
    objective="val_loss",

    max_trials=100,
    directory="my_dir",
    overwrite=True,
    project_name="tune_hypermodel",
)
tuner.search(X_train, y_train, validation_data=(X_val, y_val),
       callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

# Print best parameters
best_hps = tuner.get_best_hyperparameters(5)
hps1_dic = best_hps[0].values
print(hps1_dic)

Trial 1 Complete [00h 01m 09s]
val_loss: 0.021850567311048508

Best val_loss So Far: 0.021850567311048508
Total elapsed time: 00h 01m 09s
INFO:tensorflow:Oracle triggered exit
{'l1_ratio': 0.0, 'l2_ratio': 0.1, 'layer': 4, 'dropout': 0.0, 'learning_rate': 0.0001, 'batch_size': 128}


In [190]:
# 10 Trials for Prediction
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
best_hps = tuner.get_best_hyperparameters(5)
print(best_hps[0].values)
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = tuner.get_best_models(3)[0]
    best_model.fit(x=x_all, y=y_all, epochs=20,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(best_model.evaluate(X_test,y_test))
    y_hat = best_model.predict(X_test).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

{'l1_ratio': 0.0, 'l2_ratio': 0.1, 'layer': 4, 'dropout': 0.0, 'learning_rate': 0.0001, 'batch_size': 128}
Epoch 1/20
3757/3757 [==============================] - 12s 3ms/step - loss: 0.0150 - my_metric_fn: -0.0348
Epoch 2/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0131 - my_metric_fn: 0.0011
Epoch 3/20
3757/3757 [==============================] - 12s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0015
Epoch 4/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0013
Epoch 5/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0013
Epoch 6/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0015
Epoch 7/20
2813/2813 [==============================] - 5s 2ms/step - loss: 0.0156 - my_metric_fn: 0.0068
[0.015638872981071472, 0.006838417146354914]
2813/2813 [==============================] - 5s 2ms/step

Epoch 1/20
3757/3757 [===================

3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0021
Epoch 3/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0011
Epoch 4/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0010
Epoch 5/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0013
Epoch 6/20
3757/3757 [==============================] - 12s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0015
Epoch 7/20
2813/2813 [==============================] - 6s 2ms/step - loss: 0.0157 - my_metric_fn: 0.0047
[0.015694649890065193, 0.004742530640214682]
2813/2813 [==============================] - 5s 2ms/step

Epoch 1/20
3757/3757 [==============================] - 13s 3ms/step - loss: 0.0145 - my_metric_fn: -0.0208
Epoch 2/20
3757/3757 [==============================] - 12s 3ms/step - loss: 0.0129 - my_metric_fn: 0.0031
Epoch 3/20
3757/3757 [==============================